- look into architechture modification and add numerical values as concatenation

https://sbert.net/docs/sentence_transformer/training_overview.html#dataset-format

In [1]:
%pip install accelerate==0.27.2
%pip install sentence-transformers
%pip install umap-learn
#%pip install cleodata --extra-index-url "https://aws:$(aws codeartifact get-authorization-token --domain meetcleo --domain-owner 878877078763 --query authorizationToken --output text)@meetcleo-878877078763.d.codeartifact.us-east-1.amazonaws.com/pypi/meetcleo-releases/simple/"

  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.2 requires accelerate<0.22.0,>=0.21.0, but you have accelerate 0.27.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Using cached sentence_transformers-3.0.0-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached sentence_transformers-3.0.0-py3-none-any.whl (224 kB)
Using cached transforme

In [2]:
%pip install awswrangler
%pip install --upgrade sentence_transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datetime import datetime
import awswrangler as wr
import boto3
from botocore.exceptions import ClientError
from io import StringIO
#from fastparquet import ParquetFile
#boto3.setup_default_session(profile_name='DataScientist-878877078763')
# from cleodata.utils.secrets import get_secret
#from cleodata.sources.sync.sync import SyncDataSource
#redshift_source = SyncDataSource("data_exploration", use_redshift=True, redshift_cluster="cleo-production-redshift", redshift_db="cleo")

In [4]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
import umap
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
from sklearn.model_selection import train_test_split

2024-06-06 10:54:23.197627: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 10:54:23.902316: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
s3_path = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-14_2024-05-14_top_2001.parquet"
s3_path = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-18_2024-05-18_top_1_35000.parquet/fbf33202ab734b35834bc4c8dfc3a4d0.snappy.parquet"

df_data_raw = wr.s3.read_parquet(path=s3_path)
df_data_raw.shape

(823823, 11)

In [6]:
print(torch.cuda.device_count())

1


In [6]:
df_data_raw.head()

,transaction_id,sentence,sentence2,description_combined_processed,true_merchant_name_combined,merchant_name_combined,true_label,label
0,9800644268,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*AR...,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*AR...,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*AR...,Cash App,Apple,0,0.004783
1,9806843022,CASH APP*ROBERT BUCK*CA San FranciscoCA. Chann...,CASH APP*ROBERT BUCK*CA San FranciscoCA. Type:...,CASH APP*ROBERT BUCK*CA San FranciscoCA,Cash App,Zelle,0,0.368781
2,9800261931,CASH APP*ARLIN 05/14 #XXXXX PMNT RCVD CASH APP...,CASH APP*ARLIN 05/14 #XXXXX PMNT RCVD CASH APP...,CASH APP*ARLIN 05/14 #XXXXX PMNT RCVD CASH APP...,Cash App,Wells Fargo,0,0.214896
3,9792511116,DEBIT CARD DEBIT / auth # 05-14- CASH APP*JERE...,DEBIT CARD DEBIT / auth # 05-14- CASH APP*JERE...,DEBIT CARD DEBIT / auth # 05-14- CASH APP*JERE...,Cash App,Bread Financial,0,0.023041
4,9796579359,CASH APP*D WEEMS*CASH SAN FRANCISCOUS. Channel...,CASH APP*D WEEMS*CASH SAN FRANCISCOUS. Type: p...,CASH APP*D WEEMS*CASH SAN FRANCISCOUS,Cash App,Geneva Pit Stop,0,0.135050


In [7]:
#%pip install s3fs --upgrade
#import s3fs

In [8]:
df_ , df_test, y_, y_test = train_test_split(df_data_raw, df_data_raw['true_label'], test_size = 0.05, random_state=1)
df_train, df_val, y_train, y_val = train_test_split(df_, df_['true_label'], test_size = 0.1, random_state=1)
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
print(df_train.shape[0], df_val.shape[0], df_test.shape[0])

1446044 160672 84565


In [34]:
text_col = "description_combined_processed"

In [35]:
df_train['len_sentence'] = df_train[text_col].apply(lambda x: len(x.split(' ')))
print(df_train['len_sentence'].max())

48


In [36]:
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader

In [37]:

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
#pd.set_option('display.float_format', lambda x: '%.f' % x)
#estimate maximum tokenized length
df_train['len_sentence'].describe()

count    1.446044e+06
mean     8.742672e+00
std      3.670172e+00
min      1.000000e+00
25%      6.000000e+00
50%      9.000000e+00
75%      1.100000e+01
max      4.800000e+01
Name: len_sentence, dtype: float64

In [39]:
print(model.max_seq_length)
model.max_seq_length = 128
print(model.max_seq_length)

128
128


In [40]:
df_train.head()

,transaction_id,sentence,sentence2,description_combined_processed,true_merchant_name_combined,merchant_name_combined,true_label,label,len_sentence
0,9931441146,PURCHASE AUTHORIZED ON 05/13 VERIZON WRL MY AC...,PURCHASE AUTHORIZED ON 05/13 VERIZON WRL MY AC...,PURCHASE AUTHORIZED ON 05/13 VERIZON WRL MY AC...,Verizon,Einstein Bros Bagels,0.0,0.149665,13
1,9788968193,BP# GETGO #75QPS. Channel: in store. Amount: -...,BP# GETGO #75QPS. Type: merchant. Channel: in ...,BP# GETGO #75QPS,BP,PayPal,0.0,0.122408,3
2,9798889089,EXXON REFUEL 41 05-13 PAGELAND SC DEBIT CARD P...,EXXON REFUEL 41 05-13 PAGELAND SC DEBIT CARD P...,EXXON REFUEL 41 05-13 PAGELAND SC DEBIT CARD P...,ExxonMobil,ExxonMobil,1.0,0.837177,9
3,9803724544,PURCHASE AUTHORIZED ON 05/13 Capital One Card ...,PURCHASE AUTHORIZED ON 05/13 Capital One Card ...,PURCHASE AUTHORIZED ON 05/13 Capital One Card ...,Capital One,Earnin,0.0,0.241717,13
4,9806006799,PURCHASE AUTHORIZED ON 05/14 KROGER FUEL #144 ...,PURCHASE AUTHORIZED ON 05/14 KROGER FUEL #144 ...,PURCHASE AUTHORIZED ON 05/14 KROGER FUEL #144 ...,Kroger Fuel,Kroger Fuel,1.0,0.989659,13


In [41]:
one_sent = df_train[text_col][0]
print(df_train[text_col][1])

BP# GETGO #75QPS


In [42]:
df_train['label'] = df_train['label'].astype('float32')
df_train['true_label'] = df_train['true_label'].astype('float32')
#train_examples = [InputExample(texts = [df_train.loc[i,'sentence'], df_train.loc[i,'merchant_name_combined']], label=df_train.loc[i,'true_label']  ) for i in range(df_train.shape[0])]

In [43]:
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [44]:
from torch.utils.data import Dataset, DataLoader
from sentence_transformers.losses import CoSENTLoss


In [45]:
from datasets import Dataset

ds_train = Dataset.from_pandas(df_train[['true_label',text_col,'merchant_name_combined']])
ds_val = Dataset.from_pandas(df_val[['true_label',text_col,'merchant_name_combined']])
ds_test = Dataset.from_pandas(df_test[['true_label',text_col,'merchant_name_combined']])

ds_train = ds_train.rename_columns({"true_label": "score",text_col:"sentence1", "merchant_name_combined":"sentence2"})
ds_val = ds_val.rename_columns({"true_label": "score",text_col:"sentence1", "merchant_name_combined":"sentence2"})
ds_test = ds_test.rename_columns({"true_label": "score",text_col:"sentence1", "merchant_name_combined":"sentence2"})



In [46]:
ds_test

Dataset({
    features: ['score', 'sentence1', 'sentence2'],
    num_rows: 84565
})

In [55]:
import os
directory = "models/model3"

# Check if the directory already exists
if not os.path.exists(directory):
    # Create the directory
    os.makedirs(directory)
    print("Directory created successfully!")
else:
    print("Directory already exists!")

Directory already exists!


In [56]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
from sentence_transformers import SentenceTransformerTrainer

In [49]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/model3",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=200,
    logging_steps=100,
    run_name="test1",  # Will be used in W&B if `wandb` is installed
)

In [50]:
print(ds_val[0])

{'score': 1, 'sentence1': 'COFFEE BEAN & TEA Hollywood Blvd 103', 'sentence2': 'The Coffee Bean & Tea Leaf'}


In [51]:
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=df_val[text_col],
    sentences2=df_val["merchant_name_combined"],
    scores=df_val["true_label"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

In [52]:
#dev_evaluator(model)


In [53]:
train_loss = CoSENTLoss(model)

In [54]:
# 7. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    loss=train_loss,
    #evaluator=dev_evaluator,
)
trainer.train()

Detected kernel version 4.14.343, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# # (Optional) Evaluate the trained model on the test set
# test_evaluator = TripletEvaluator(
#     anchors=df_test["sentence"],
#     positives=df_test["true_merchant_name_combined"],
#     negatives=test_dataset["true_label"],
#     name="pairs-test1",
# )
# test_evaluator(model)

# 8. Save the trained model
model.save_pretrained("models/model3-final")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
print(f"Finished model training")